In [2]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
import sys
sys.path.insert(0, os.path.abspath("../.."))
from utils import time, data_processor, constants

In [3]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu_27_countries = constants.eu
balkans = constants.balkans

In [4]:
    # SELECT 
    #     bvd_id_number,
    #     operating_revenue_turnover_,
    #     number_of_employees,
    #     costs_of_goods_sold,
    #     material_costs,
    #     added_value,
    #     total_assets,
    #     profit_margin_,
    #     operating_revenue_turnover_,
    #     EXTRACT(YEAR FROM closing_date) AS year  

In [5]:
# key_financials_detail_path = "..\\..\\data_raw\deatailed_financials\\orbis_monthly_industry_global_financials_and_ratios_eur\\"
key_financials_detail_path = "Z:\\dati_moody\\data_raw\\detailed_financials\\orbis_monthly_industry_global_financials_and_ratios_eur\\"

def fetch_data_for_country(country_iso_code):
    query = f"""
    SELECT 
        bvd_id_number,
        operating_revenue_turnover_,
        number_of_employees,
        total_assets,
        profit_margin_,
        costs_of_goods_sold,
        gross_profit,
        material_costs,
        cash_flow,
        added_value,
        p_l_before_tax, 
        roe_using_p_l_before_tax_, 
        roce_using_p_l_before_tax_, 
        EXTRACT(YEAR FROM closing_date) AS year  
    FROM 
        parquet_scan('{key_financials_detail_path}*.parquet')
    WHERE
        CAST(bvd_id_number AS STRING) LIKE '{country_iso_code}%'
    ORDER BY 
        bvd_id_number, year
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()

    return df


In [6]:
import os

output_path = "..\\..\\data_processed\\key_financials_detailed_processed\\"

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

for country in ['RU']:
    df = fetch_data_for_country(country)
    df.to_parquet(f"{output_path}key_financials_detailed_{country}.parquet")
    print(f"Saved: {output_path}key_financials_detailed_{country}.parquet")


Saved: ..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_RU.parquet
